# Notes
- Ensure ACCOUNTNO column in davis_extended_simplified.csv as been converted to text using excel
- Housing Unit Inventory: Can compare number of units by type to city population?

In [1]:
import arcpy
import os
import pandas as pd
from arcgis import GIS
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [2]:
# tag_expression and code block for tagging parcels
tag_expression = "createTag(!TAG!, '{}')"
tag_codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

In [3]:
# inputs
taz_shp = '.\\Inputs\\TAZ.shp'
parcels = '.\\Inputs\\Davis_County_LIR_Parcels.gdb\\Parcels_Davis_LIR_UTM12'

parcels_old = '.\\Inputs\\Parcels_Davis.shp'

In [4]:
# create output gdb
outputs = '.\\Outputs'
gdb = os.path.join(outputs, "davis.gdb")
if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs, "davis.gdb")
    
scratch = os.path.join(outputs, "scratch.gdb")
if not arcpy.Exists(scratch):
    arcpy.CreateFileGDB_management(outputs, "scratch.gdb")

In [5]:
# get address points without base address point
address_pts_lyr = arcpy.MakeFeatureLayer_management('.\\Inputs\\AddressPoints_Davis.gdb\\address_points_davis','address_pts_lyr')
query = (''' PtType <> 'BASE ADDRESS' ''')
arcpy.SelectLayerByAttribute_management(address_pts_lyr, 'NEW_SELECTION', query)
address_pts_no_base = arcpy.FeatureClassToFeatureClass_conversion(address_pts_lyr, scratch, '_00_address_pts_no_base')

In [6]:
# select parcels within modeling area
parcels_layer = arcpy.MakeFeatureLayer_management(parcels, 'parcels') 
arcpy.SelectLayerByLocation_management(parcels_layer, 'HAVE_THEIR_CENTER_IN', taz_shp)
parcels_for_modeling = arcpy.FeatureClassToFeatureClass_conversion(parcels_layer, gdb, '_01_parcels_for_modeling')

# recalc acreage 
arcpy.CalculateField_management(parcels_for_modeling, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

# create the main layer
parcels_for_modeling_layer = arcpy.MakeFeatureLayer_management(parcels_for_modeling, 'parcels_for_modeling_lyr') 

In [7]:
################################
# Dissolve duplicate parcel ids
################################

# dissolve on parcel id,summarizing attributes in various ways
parcels_dissolved = arcpy.management.Dissolve(parcels_for_modeling_layer, os.path.join(scratch, '_00_parcels_dissolved'), 
                                              "PARCEL_ID", [['PARCEL_ID','COUNT'],['TAXEXEMPT_TYPE','FIRST'],
                                                            ['TOTAL_MKT_VALUE','SUM'],['LAND_MKT_VALUE','SUM'],
                                                            ['PARCEL_ACRES','SUM'],['PROP_CLASS', 'FIRST'],
                                                            ['PRIMARY_RES','FIRST'],['HOUSE_CNT','MAX'],
                                                            ['BLDG_SQFT','SUM'],['FLOORS_CNT','MAX'],['BUILT_YR','FIRST'],
                                                            ['EFFBUILT_YR','FIRST']], 
                                              "MULTI_PART", "DISSOLVE_LINES")

# rename columns
parcels_dissolved_sdf = pd.DataFrame.spatial.from_featureclass(parcels_dissolved)
parcels_dissolved_sdf.columns = ['OBJECTID', 'PARCEL_ID', 'COUNT_PARCEL_ID', 'TAXEXEMPT_TYPE','TOTAL_MKT_VALUE', 
                                 'LAND_MKT_VALUE', 'PARCEL_ACRES','PROP_CLASS', 'PRIMARY_RES', 
                                 'HOUSE_CNT','BLDG_SQFT', 'FLOORS_CNT', 'BUILT_YR','EFFBUILT_YR', 
                                 'SHAPE']

# remove parcels without parcel ids
parcels_dissolved_sdf = parcels_dissolved_sdf[parcels_dissolved_sdf['PARCEL_ID'].isnull() == False].copy()

In [8]:
# load extended descriptions
def add_leading_zeroes(parcel_id_str):
    if len(parcel_id_str) == 8:
        return "0{}".format(str(parcel_id_str))
    if len(parcel_id_str) == 7:
        return "00{}".format(str(parcel_id_str))
    else:
        return parcel_id_str
    
# Load Extended Descriptions - be sure to format ACCOUNTNO column as text in excel first
ext_desc = pd.read_csv(r".\Inputs\davis_extended_simplified.csv", dtype={'ACCOUNTNO':str})

# format account numbers so that they are all 9 characters long
ext_desc['ACCOUNTNO'] = ext_desc['ACCOUNTNO'].astype(str).map(add_leading_zeroes)
ext_desc = ext_desc[['ACCOUNTNO','des_all','class']].copy()

# parcels_sdf = pd.DataFrame.spatial.from_featureclass(parcels_for_modeling_layer)
parcels_sdf = parcels_dissolved_sdf.merge(ext_desc, left_on='PARCEL_ID',right_on='ACCOUNTNO',how='left')
parcels_sdf.spatial.to_featureclass(location=os.path.join(scratch, '_01_parcels_extended'),sanitize_columns=False)

updated_parcels = os.path.join(scratch, '_01_parcels_extended')
parcels_for_modeling_layer = arcpy.MakeFeatureLayer_management(updated_parcels, 'parcels_for_modeling_lyr')

C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
# add a tag field for parcel type
arcpy.AddField_management(parcels_for_modeling_layer, 'TYPE_WFRC', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'SUBTYPE_WFRC', 'TEXT')

# add REMM fields
arcpy.AddField_management(parcels_for_modeling_layer, 'max_dua', 'FLOAT')
arcpy.AddField_management(parcels_for_modeling_layer, 'max_far', 'FLOAT')
arcpy.AddField_management(parcels_for_modeling_layer, 'max_height', 'FLOAT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type1', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type2', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type3', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type4', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type5', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type6', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type7', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'type8', 'TEXT')
arcpy.AddField_management(parcels_for_modeling_layer, 'agriculture', 'SHORT')
arcpy.AddField_management(parcels_for_modeling_layer, 'basebldg', 'SHORT')
arcpy.AddField_management(parcels_for_modeling_layer, 'NoBuild', 'SHORT') # REMM doesn't use this
arcpy.AddField_management(parcels_for_modeling_layer, 'redev_friction', 'FLOAT')
arcpy.AddField_management(parcels_for_modeling_layer, 'building_type_id', 'SHORT')


# count parts and rings
arcpy.AddField_management(parcels_for_modeling_layer, 'PARTS', 'LONG')
arcpy.AddField_management(parcels_for_modeling_layer, 'RINGS', 'LONG')

fields = ["shape@", 'PARTS', 'RINGS']
with arcpy.da.UpdateCursor(parcels_for_modeling_layer, fields) as cursor:
    for row in cursor:
        shape = row[0]
        parts = shape.partCount
        rings = shape.boundary().partCount   
        row[1] = parts
        row[2] = rings
        cursor.updateRow(row)
        
# calculate default for basebldg field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='basebldg', expression="0", expression_type="PYTHON3")

# calculate default for parcel type fields
arcpy.CalculateField_management(parcels_for_modeling_layer, field='type1', expression="'f'", expression_type="PYTHON3")
arcpy.CalculateField_management(parcels_for_modeling_layer, field='type2', expression="'f'", expression_type="PYTHON3")
arcpy.CalculateField_management(parcels_for_modeling_layer, field='type3', expression="'f'", expression_type="PYTHON3")
arcpy.CalculateField_management(parcels_for_modeling_layer, field='type4', expression="'f'", expression_type="PYTHON3")
arcpy.CalculateField_management(parcels_for_modeling_layer, field='type5', expression="'f'", expression_type="PYTHON3")
arcpy.CalculateField_management(parcels_for_modeling_layer, field='type6', expression="'f'", expression_type="PYTHON3")
arcpy.CalculateField_management(parcels_for_modeling_layer, field='type7', expression="'f'", expression_type="PYTHON3")
arcpy.CalculateField_management(parcels_for_modeling_layer, field='type8', expression="'f'", expression_type="PYTHON3")

# delete parcels with empty geometry
query = (""" PARTS =  0""")
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# get a count of all parcels
count_all = arcpy.GetCount_management(parcels_for_modeling_layer)
print("# initial parcels in modeling area:\n {}".format(count_all))


# initial parcels in modeling area:
 113616


In [10]:
###############
# Common Areas
###############

common_areas = r'.\Inputs\Common_Areas.gdb\Common_Areas_Reviewed'
common_areas_lyr = arcpy.MakeFeatureLayer_management(common_areas, 'common_areas_lyr') 

# open_space
query = """ TYPE_WFRC IN ('open_space') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_open_space = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02b_ca_open_space')

# utility
query = """ TYPE_WFRC IN ('utility') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_utility = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02c_ca_utility')

# retail
query = """ TYPE_WFRC IN ('retail') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_retail = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02d_ca_retail')

# industrial
query = """ TYPE_WFRC IN ('industrial') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_industrial = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02e_ca_industrial')

# office
query = """ TYPE_WFRC IN ('office') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_office = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02f_ca_office')

# pud
query = """ SUBTYPE_WFRC IN ('pud') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_pud = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02g_ca_pud')

# multi_family
query = """ TYPE_WFRC IN ('multi_family') """
arcpy.SelectLayerByAttribute_management(common_areas_lyr, 'NEW_SELECTION', query)
ca_multi_family = arcpy.FeatureClassToFeatureClass_conversion(common_areas_lyr, scratch, '_02e_ca_multi_family')


In [11]:
###############
# PUD
###############

tag = "single_family"
tag2 = "pud"

# parcels that are contained by condo common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="INTERSECT", 
                                       select_features=ca_pud, selection_type='NEW_SELECTION')

# convert condo parcels that are contained by common areas into centroids
pud_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(scratch, '_03a_pud_centroids'), "INSIDE")

# recalc acreage
arcpy.CalculateField_management(ca_pud, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#==================================================
# summarize units attributes within pud areas
#==================================================

# use spatial join to summarize market value & acreage
target_features = ca_pud
join_features = pud_centroids
output_features = os.path.join(scratch, "_03a_pud_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('LAND_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# # parcel acres
# fieldindex = fieldmappings.findFieldMapIndex('PARCEL_ACRES')
# fieldmap = fieldmappings.getFieldMap(fieldindex)
# fieldmap.mergeRule = 'Sum'
# fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# year
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

# calculate the type field
arcpy.CalculateField_management(oug_sj, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

arcpy.CalculateField_management(oug_sj, field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")

# rename join_count
arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!",
                                expression_type="PYTHON3")

arcpy.DeleteField_management(oug_sj, "Join_Count")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = oug_sj 
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_pud")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECT")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

#################################
# WRAP-UP
#################################

# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=oug_sj2, selection_type='NEW_SELECTION')
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=oug_sj2, selection_type='ADD_TO_SELECTION')

count_type = arcpy.GetCount_management(parcels_for_modeling_layer)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="1",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

2655 "single_family" parcels were selected.
110961 parcels remain...


In [12]:
###############
# multi family
###############

tag = "multi_family"

# parcels that are contained by condo common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="INTERSECT", 
                                       select_features=ca_multi_family, selection_type='NEW_SELECTION')

# convert condo parcels that are contained by common areas into centroids
mf_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(scratch, '_03a_mf_centroids'), "INSIDE")

# recalc acreage
arcpy.CalculateField_management(ca_multi_family, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#==================================================
# summarize units attributes within pud areas
#==================================================

# use spatial join to summarize market value & acreage
target_features = ca_multi_family
join_features = mf_centroids
output_features = os.path.join(scratch, "_03b_mf_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# # parcel acres
# fieldindex = fieldmappings.findFieldMapIndex('PARCEL_ACRES')
# fieldmap = fieldmappings.getFieldMap(fieldindex)
# fieldmap.mergeRule = 'Sum'
# fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# year
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

# calculate the type field
arcpy.CalculateField_management(oug_sj, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# rename join_count
arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!",
                                expression_type="PYTHON3")

arcpy.DeleteField_management(oug_sj, "Join_Count")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = oug_sj 
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_multi_family")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECT")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

#################################
# WRAP-UP
#################################

# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=oug_sj2, selection_type='NEW_SELECTION')
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=oug_sj2, selection_type='ADD_TO_SELECTION')

count_type = arcpy.GetCount_management(parcels_for_modeling_layer)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="2",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

9425 "multi_family" parcels were selected.
101536 parcels remain...


In [13]:
#####################
# industrial commons
#####################

tag = "industrial"

# parcels that are contained by industrial common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="INTERSECT", 
                                       select_features=ca_industrial, selection_type='NEW_SELECTION')

# convert condo parcels that are contained by common areas into centroids
mf_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(scratch, '_03a_industrial_centroids'), "INSIDE")

# recalc acreage
arcpy.CalculateField_management(ca_industrial, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#==================================================
# summarize units attributes within pud areas
#==================================================

# use spatial join to summarize market value & acreage
target_features = ca_industrial
join_features = mf_centroids
output_features = os.path.join(scratch, "_03c_industrial_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# # parcel acres
# fieldindex = fieldmappings.findFieldMapIndex('PARCEL_ACRES')
# fieldmap = fieldmappings.getFieldMap(fieldindex)
# fieldmap.mergeRule = 'Sum'
# fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# year
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

# calculate the type field
arcpy.CalculateField_management(oug_sj, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# rename join_count
arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!",
                                expression_type="PYTHON3")

arcpy.DeleteField_management(oug_sj, "Join_Count")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = oug_sj 
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_industrial_commons")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECT")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

#################################
# WRAP-UP
#################################

# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=oug_sj2, selection_type='NEW_SELECTION')
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=oug_sj2, selection_type='ADD_TO_SELECTION')

count_type = arcpy.GetCount_management(parcels_for_modeling_layer)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="3",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

401 "industrial" parcels were selected.
101135 parcels remain...


In [14]:
#####################
# office commons
#####################

tag = "office"

# parcels that are contained by industrial common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="INTERSECT", 
                                       select_features=ca_office, selection_type='NEW_SELECTION')

# convert condo parcels that are contained by common areas into centroids
office_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(scratch, '_03a_office_centroids'), "INSIDE")

# recalc acreage
arcpy.CalculateField_management(ca_office, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#==================================================
# summarize units attributes within pud areas
#==================================================

# use spatial join to summarize market value & acreage
target_features = ca_office
join_features = office_centroids
output_features = os.path.join(scratch, "_03d_office_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# # parcel acres
# fieldindex = fieldmappings.findFieldMapIndex('PARCEL_ACRES')
# fieldmap = fieldmappings.getFieldMap(fieldindex)
# fieldmap.mergeRule = 'Sum'
# fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# year
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

# calculate the type field
arcpy.CalculateField_management(oug_sj, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# rename join_count
arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!",
                                expression_type="PYTHON3")

arcpy.DeleteField_management(oug_sj, "Join_Count")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = oug_sj 
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_office_commons")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECT")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

#################################
# WRAP-UP
#################################

# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=oug_sj2, selection_type='NEW_SELECTION')
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=oug_sj2, selection_type='ADD_TO_SELECTION')

count_type = arcpy.GetCount_management(parcels_for_modeling_layer)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="5",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

273 "office" parcels were selected.
100862 parcels remain...


In [15]:
#####################
# retail commons
#####################

tag = "retail"

# parcels that are contained by industrial common areas
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="INTERSECT", 
                                       select_features=ca_retail, selection_type='NEW_SELECTION')

# convert condo parcels that are contained by common areas into centroids
retail_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, 
                                                  os.path.join(scratch, '_03a_retail_centroids'), "INSIDE")

# recalc acreage
arcpy.CalculateField_management(ca_retail, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#==================================================
# summarize units attributes within pud areas
#==================================================

# use spatial join to summarize market value & acreage
target_features = ca_retail
join_features = retail_centroids
output_features = os.path.join(scratch, "_03e_retail_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# # parcel acres
# fieldindex = fieldmappings.findFieldMapIndex('PARCEL_ACRES')
# fieldmap = fieldmappings.getFieldMap(fieldindex)
# fieldmap.mergeRule = 'Sum'
# fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# year
fieldindex = fieldmappings.findFieldMapIndex('BUILT_YR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join, use 'Join_Count' for number of units
oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

# calculate the type field
arcpy.CalculateField_management(oug_sj, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# rename join_count
arcpy.CalculateField_management(oug_sj, field='parcel_count', expression="!Join_Count!",
                                expression_type="PYTHON3")

arcpy.DeleteField_management(oug_sj, "Join_Count")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = oug_sj 
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_retail_commons")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECTS")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

#################################
# WRAP-UP
#################################

# delete features from working parcels
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="HAVE_THEIR_CENTER_IN", 
                                       select_features=oug_sj2, selection_type='NEW_SELECTION')
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=oug_sj2, selection_type='ADD_TO_SELECTION')

count_type = arcpy.GetCount_management(parcels_for_modeling_layer)
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count of remaining parcels
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="4",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

93 "retail" parcels were selected.
100769 parcels remain...


In [16]:
###########
# churches
###########

query = (""" class IN ('churches') """) 
tag="churches"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="8",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

256 "churches" parcels were selected.
100513 parcels remain...


In [17]:
###############
# agriculture
###############

query = """ class IN ('agriculture') OR (PARCEL_ID IN ('090150001','090120001','090170017','110740084','110740073',
                                                       '110740086','110740090','110740084','110750135','110740080',
                                                       '110740073','110740074','121090148','121090282','121090150',
                                                       '121090147','121090322','120810103','120810096','121020066',
                                                       '121020065')) """
tag="agriculture"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="1",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

233 "agriculture" parcels were selected.
100280 parcels remain...


In [18]:
################
# single_family
################

query = (""" class IN ('single_family') """)
tag="single_family"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
single_family = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="1",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

82130 "single_family" parcels were selected.
18150 parcels remain...


In [19]:
################
# Multi-Family
################

query = (""" class IN ('multi_family', 'duplex', 'group_quarters', 'apartment', 'townhome', 'triplex-quadplex') """)
tag="multi_family2"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="!class!", expression_type="PYTHON3")

# create the feature class for the parcel type
mf2_commons = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_02_mf2_commons')

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = mf2_commons
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_multi_family2")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, match_option="INTERSECT")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

#################################
# WRAP-UP
#################################

# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(oug_sj2, field='building_type_id', expression="2",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

1679 "multi_family2" parcels were selected.
16471 parcels remain...


In [20]:
###############
# government
###############

query = (""" (class IN ('government','education'))  OR (PARCEL_ID IN ('090150001', '090120001','090170017')) """)
tag="government"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# get parcels on parks
schools = ".\\Inputs\\Utah_Schools_PreK_to_12.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="INTERSECT",
                                       select_features=schools,
                                       selection_type='ADD_TO_SELECTION')

# get parcels on hill afb
hill_afb = ".\\Inputs\\hill_afb.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="INTERSECT",
                                       select_features=hill_afb,
                                       selection_type='ADD_TO_SELECTION')


# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="6",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

832 "government" parcels were selected.
15639 parcels remain...


In [21]:
###############
# retail
###############

query = (""" class IN ('retail') """)
tag="retail"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="4",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

1072 "retail" parcels were selected.
14567 parcels remain...


In [22]:
###############
# office
###############

query = (""" class IN ('office') """)
tag="office"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="6",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

370 "office" parcels were selected.
14197 parcels remain...


In [23]:
###############
# industrial
###############

query = (""" class IN ('industrial') or PARCEL_ID IN ('060920006','060920004','060920050','060920051','060920024',
                                                      '011030008','011030005','011040006','011040005','011040003',
                                                      '011030031','011060002','011050006','011050005','011060004',
                                                      '011060005','011060004','011050010') """)
tag="industrial"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)


# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="3",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

1239 "industrial" parcels were selected.
12958 parcels remain...


In [24]:
###############
# healthcare
###############

query = (""" class IN ('healthcare') """)
tag = 'healthcare'

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# add parcels that overlap with healthcare points - currently excluding nursing homes
healthcare_facilties_lyr = arcpy.MakeFeatureLayer_management('.\\Inputs\\HealthCareFacilities.shp','healthcare_facilties_lyr') 
query = """ TYPE NOT IN ('NURSING HOME', 'ASSISTED LIVING FACILITY') """
arcpy.SelectLayerByAttribute_management(healthcare_facilties_lyr, 'NEW_SELECTION', query)
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="INTERSECT",
                                       select_features=healthcare_facilties_lyr,
                                       selection_type='ADD_TO_SELECTION')


# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate basebldg field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='basebldg', expression="1",
                                expression_type="PYTHON3")

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='building_type_id', expression="8",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

185 "healthcare" parcels were selected.
12773 parcels remain...


In [25]:
#################
# OUG - mobile home parks
#################

tag = "multi_family"
tag2 = "mobile_home_park"

# use overlay to select mobile home parks parcels
mobile_home_parks = ".\\Inputs\\Mobile_Home_Parks.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=mobile_home_parks,
                                       selection_type='NEW_SELECTION')
query= (""" class IN ('mobile_home_park') """)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)


# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='SUBTYPE_WFRC', expression="'{}'".format(tag2),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
mhp = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, scratch, '_07a_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# recalc acreage 
# arcpy.CalculateGeometryAttributes_management(mhp, [["PARCEL_ACRES", "AREA"]], area_unit='ACRES')
arcpy.CalculateField_management(mhp, "PARCEL_ACRES", """!SHAPE.area@ACRES!""", "PYTHON3")

#################################
# get count from address points
#################################

# summarize address points address_point_count "ap_count"
target_features = mhp
join_features = address_pts_no_base
output_features = os.path.join(gdb, "_02_mobile_home_park")

oug_sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           match_option="INTERSECT")

arcpy.CalculateField_management(oug_sj2, field='ap_count', expression="!Join_Count!", expression_type="PYTHON3")
arcpy.DeleteField_management(oug_sj2, "Join_Count")

# calculate basebldg field
arcpy.CalculateField_management(oug_sj2, field='basebldg', expression="1",
                                expression_type="PYTHON3")

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

88 "multi_family" parcels were selected.
12685 parcels remain...


In [26]:
###############
# open space
###############

# get parcels on parks
parks = ".\\Inputs\\ParksLocal.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=parks,
                                       selection_type='NEW_SELECTION')

# get parcels on public lands that are not military
land_ownership_lyr = arcpy.MakeFeatureLayer_management('./Inputs/UT_SITLA_Ownership_LandOwnership_WM.shp','land_ownership_lyr') 

query= (""" OWNER <> 'Private' And DESIG <> 'Military' """)
arcpy.SelectLayerByAttribute_management(land_ownership_lyr, 'NEW_SELECTION', query)

arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=land_ownership_lyr,
                                       selection_type='ADD_TO_SELECTION')

# get cemetery parcels using overlay
cemeteries_lyr = arcpy.MakeFeatureLayer_management('.\\Inputs\\Cemeteries.shp','cemeteries_lyr') 
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="INTERSECT",
                                       select_features=cemeteries_lyr,
                                       selection_type='ADD_TO_SELECTION')

# get parcels that are golf courses using overlay
golf_courses = ".\\Inputs\\GolfCourses.shp"
arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="HAVE_THEIR_CENTER_IN",
                                       select_features=golf_courses,
                                       selection_type='ADD_TO_SELECTION')

# query parcel attributes
query= (""" class IN ('open_space') """)
tag = "open_space"

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

# calculate the type field
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# calculate building_type_id field
arcpy.CalculateField_management(os.path.join(gdb, '_02_{}'.format(tag)), field='NoBuild', expression="1",
                                expression_type="PYTHON3")

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)

# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

389 "open_space" parcels were selected.
12296 parcels remain...


In [27]:
####################
# Drop small parcels
####################

query = """ Shape_Area < 375 """

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

# count the selected features
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

arcpy.DeleteFeatures_management(parcels_for_modeling_layer)
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

2822 "open_space" parcels were selected.
9474 parcels remain...


In [28]:
###############################################
# Identify roads using shape attribute ratios
###############################################

arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")

# copy shape fields
arcpy.AddField_management(parcels_for_modeling_layer, 'S_Area', 'FLOAT')
arcpy.CalculateField_management(parcels_for_modeling_layer, field='S_Area', expression="!Shape_Area!",
                                expression_type="PYTHON3")

arcpy.AddField_management(parcels_for_modeling_layer, 'S_Length', 'FLOAT')
arcpy.CalculateField_management(parcels_for_modeling_layer, field='S_Length', expression="!Shape_Length!",
                                expression_type="PYTHON3")

# get mimimum bounding rectangle by width of remaining parcels
mbrw = os.path.join(scratch, "_05_minimum_bounding_rectangle")
mbrw = arcpy.MinimumBoundingGeometry_management(parcels_for_modeling_layer, mbrw, "RECTANGLE_BY_WIDTH", 
                                                mbg_fields_option='MBG_FIELDS')

mbrw_df = pd.DataFrame.spatial.from_featureclass(mbrw)
mbrw_df = mbrw_df[['PARCEL_ID', 'MBG_Length', 'MBG_Width']].copy()
parcels_remaining_df = pd.DataFrame.spatial.from_featureclass(parcels_for_modeling_layer)
parcel_new = parcels_remaining_df.merge(mbrw_df, left_on='PARCEL_ID',right_on='PARCEL_ID', how='left')
parcel_new['Len_Wid_Ratio'] = parcel_new['MBG_Length']/parcel_new['MBG_Width']
parcel_new['Len_Area_Ratio'] = parcel_new['MBG_Length']/parcel_new['S_Area']
parcel_new['Perim_Area_Ratio'] = parcel_new['S_Length']/parcel_new['S_Area']

parcel_new.spatial.to_featureclass(location=os.path.join(scratch, "_06_parcels_extended"),sanitize_columns=False)


parcels_for_modeling_layer = arcpy.MakeFeatureLayer_management(os.path.join(scratch, "_06_parcels_extended"), 
                                                               'parcels_for_modeling_lyr')

query = """ Len_Wid_Ratio >= 4.5 And Len_Area_Ratio >= 0.025 And Perim_Area_Ratio >= 0.055 """
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
count_type = arcpy.GetCount_management(parcels_for_modeling_layer)

tag = "roads"
arcpy.CalculateField_management(parcels_for_modeling_layer, field='TYPE_WFRC', expression="'{}'".format(tag),
                                expression_type="PYTHON3")
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

# delete features from working parcels
parcels_for_modeling_layer = arcpy.DeleteFeatures_management(parcels_for_modeling_layer)

# count remaining features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, "CLEAR_SELECTION")
count_remaining = arcpy.GetCount_management(parcels_for_modeling_layer)
# message
print('{} "{}" parcels were selected.\n{} parcels remain...'.format(count_type, tag, count_remaining))

1369 "roads" parcels were selected.
8105 parcels remain...


In [29]:
#################
# undevelopable
#################

tag = 'undevelopable'

############
# Water
############

water_lakes = '.\Inputs\water_lakes.shp'
water_lakes_lyr = arcpy.MakeFeatureLayer_management(water_lakes, 'water_lakes_lyr') 
query = """ ftype_text NOT IN ('Swamp/Marsh') """
arcpy.SelectLayerByAttribute_management(water_lakes_lyr, 'NEW_SELECTION', query)

arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer,overlap_type="INTERSECT",
                                       select_features=water_lakes_lyr,
                                       selection_type='NEW_SELECTION')

########################
# tax_exempt parcels
########################

query = """ TAXEXEMPT_TYPE IN ('YES', 'Yes') """

# select the features
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'ADD_TO_SELECTION', query)


# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))

<Result '.\\Outputs\\davis.gdb\\_02_undevelopable'>

In [30]:
###########################
# vacant, developable land
###########################

# query = """ PROP_CLASS = 'Vacant Land' """
# arcpy.SelectLayerByAttribute_management(water_lakes_lyr, 'NEW_SELECTION', query)
tag = 'vacant'

# create the feature class for the parcel type
arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_{}'.format(tag))



<Result '.\\Outputs\\davis.gdb\\_02_vacant'>

In [31]:
# # export remaining parcels
# arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_09_unclassed_parcels')

In [33]:
###################
# Merge the parcels back together
###################

arcpy.env.workspace = gdb
featureclasses = arcpy.ListFeatureClasses()
merged_parcels = arcpy.Merge_management(featureclasses, os.path.join(gdb, 'merged_parcels'), add_source='ADD_SOURCE_INFO')
merged_parcels_lyr = arcpy.MakeFeatureLayer_management(merged_parcels, 'gflu_lyr') 


In [35]:
############### 
# Parcel Types
###############


# set land use type on parcels using glfu
gflu = '.\Inputs\FutureLandUse2020.gdb\FutureLandUse2020'
gflu_lyr = arcpy.MakeFeatureLayer_management(gflu, 'gflu_lyr') 





##################
# Type 1 (SF)
##################

# query for land use type
query = """ GenLUType IN ('Any Development','Any Residential', 'Mixed Use SF', 'Residential SF', 'Residential SF/Retail') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type1', expression="'t'".format(tag),
                                expression_type="PYTHON3")


##################
# Type 2 (MF)
##################

# query for land use type
query = """ GenLUType IN ('Any Residential', 'Any Development', 'Industrial/Mixed Use MF', 'Mixed Use', 
                          'Residential MF/Office', 'Residential MF', 'Mixed Use MF', 'Mixed Use SF', 'Residential/Office', 
                          'Residential/Retail') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type2', expression="'t'".format(tag),
                                expression_type="PYTHON3")

#######################
# Type 3 (Industrial)
#######################

# query for land use type
query = """ GenLUType IN ('Industrial', 'Any Development', 'Industrial/Mixed Use MF') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type3', expression="'t'".format(tag),
                                expression_type="PYTHON3")

#######################
# Type 4 (Retail)
#######################

# query for land use type
query = """ GenLUType IN ('Any Development', 'Industrial/Retail', 'Retail', 'Retail/Office', 'Residential/Retail', 
                          'Residential SF/Retail') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type4', expression="'t'".format(tag),
                                expression_type="PYTHON3")


#######################
# Type 5 (Office)
#######################

# query for land use type
query = """ GenLUType IN ('Retail/Office', 'Office', 'Any Commercial', 'Residential/Office', 'Residential MF/Office') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type5', expression="'t'".format(tag),
                                expression_type="PYTHON3")

####################################
# Type 6 (Government and Education)
###################################

# query for land use type
query = """ GenLUType IN ('Any Development', 'Government/Education') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type6', expression="'t'".format(tag),
                                expression_type="PYTHON3")


####################################
# Type 7 (Mixed Use)
###################################

# query for land use type
query = """ GenLUType IN ('Any Development', 'Industrial/Mixed Use MF', 'Mixed Use', 'Mixed Use MF', 'Mixed Use SF', 
                          'Residential MF/Office', 'Residential SF/Retail', 'Residential/Office', 'Residential/Retail', 
                          'Retail/Office') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type7', expression="'t'".format(tag),
                                expression_type="PYTHON3")

####################################
# Type 8 (Other)
###################################

# query for land use type
query = """ GenLUType IN ('Any Development', 'Industrial/Retail', 'Retail', 'Retail/Office', 'Residential/Retail', 
                          'Residential SF/Retail') """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type8', expression="'t'".format(tag),
                                expression_type="PYTHON3")

#################################
# Undevelopable
#################################

# query for land use type
query = """ GenLUType IN ('NoBuild') or NoBuild = 1 """
arcpy.SelectLayerByAttribute_management(gflu_lyr, 'NEW_SELECTION', query)

# select the parcels
arcpy.SelectLayerByLocation_management(in_layer=merged_parcels_lyr,overlap_type="INTERSECT",
                                       select_features=gflu_lyr,
                                       selection_type='NEW_SELECTION')

arcpy.CalculateField_management(merged_parcels_lyr, field='type1', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type2', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type3', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type4', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type5', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type6', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type7', expression="'f'".format(tag),
                                expression_type="PYTHON3")
arcpy.CalculateField_management(merged_parcels_lyr, field='type8', expression="'f'".format(tag),
                                expression_type="PYTHON3")

ExecuteError: ERROR 000358: Invalid expression
Failed to execute (SelectLayerByAttribute).


In [ ]:
####################################
# Eject buildings from parcel data
####################################

# create new REMM parcel IDs

# copy table and only keep rows if basebldg = 1 

# generate building id

# keep fields [building_sqft, building_type_id, parcel_id, residential_units, stories, unit_price, year_built]

# add empty fields [non_residential_sqft, res_price_per_sqft, job_spaces]

In [ ]:
################################
# Create Housing Unit Inventory
################################

# create paths
res_files = ['_02_pud', '_02_single_family', '_02_mobile_home_park', '_02_multi_family', '_02_multi_family2']
res_files = [os.path.join(gdb, res_file) for res_file in res_files]

# merge features
rf_merged = arcpy.Merge_management(res_files, os.path.join(scratch, '_10_Housing_Unit_Inventory'), add_source='ADD_SOURCE_INFO')

cities = r'.\Inputs\Cities.shp' 
target_features = rf_merged
join_features = cities
output_features = os.path.join(gdb, '_04_Housing_Unit_Inventory')

rf_merged = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           match_option="HAVE_THEIR_CENTER_IN")


# rf_subset = ".\Example_Clearfield_Layton\Example.gdb\HUI_Davis_Subset"
# rf_merged = rf_subset


# convert to dataframe and export
rf_merged_df = pd.DataFrame.spatial.from_featureclass(rf_merged)
rf_merged_df = rf_merged_df[['OBJECTID','Type_WFRC', 'SUBTYPE_WFRC', 'PARCEL_ID','COUNT_PARCEL_ID', 'TOTAL_MKT_VALUE', 
                             'LAND_MKT_VALUE', 'PARCEL_ACRES', 'HOUSE_CNT', 'parcel_count', 'ap_count', 'BLDG_SQFT', 
                             'FLOORS_CNT','BUILT_YR', 'des_all', 'NAME']].copy()

rf_merged_df = rf_merged_df.rename(columns={"NAME": "CITY"})

# convert unit count columns to int
rf_merged_df.loc[(rf_merged_df['HOUSE_CNT'].isnull() == True ), 'HOUSE_CNT'] = 0
rf_merged_df['HOUSE_CNT'] = rf_merged_df['HOUSE_CNT'].astype(int)
rf_merged_df.loc[(rf_merged_df['ap_count'].isnull() == True ), 'ap_count'] = 0
rf_merged_df['ap_count'] = rf_merged_df['ap_count'].astype(int)


# create new count field and calculate
rf_merged_df['UNIT_COUNT'] = rf_merged_df['ap_count']

# fix single family (non-pud)
rf_merged_df.loc[(rf_merged_df['UNIT_COUNT'] == 0) & 
                 (rf_merged_df['SUBTYPE_WFRC'] == 'single_family'), 
                 'UNIT_COUNT'] = 1

# fix duplex
rf_merged_df.loc[(rf_merged_df['SUBTYPE_WFRC'] == 'duplex'), 'UNIT_COUNT'] = 2

# fix triplex-quadplex
rf_merged_df.loc[(rf_merged_df['UNIT_COUNT'] < rf_merged_df['HOUSE_CNT']) & 
                 (rf_merged_df['SUBTYPE_WFRC'] == 'triplex-quadplex'), 
                 'UNIT_COUNT'] = rf_merged_df['HOUSE_CNT']

# calculate the decade
rf_merged_df['BUILT_DECADE'] = 'NA'
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1840) & (rf_merged_df['BUILT_YR'] < 1850), 'BUILT_DECADE'] = "1840's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1850) & (rf_merged_df['BUILT_YR'] < 1860), 'BUILT_DECADE'] = "1850's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1860) & (rf_merged_df['BUILT_YR'] < 1870), 'BUILT_DECADE'] = "1860's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1870) & (rf_merged_df['BUILT_YR'] < 1880), 'BUILT_DECADE'] = "1870's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1880) & (rf_merged_df['BUILT_YR'] < 1890), 'BUILT_DECADE'] = "1880's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1890) & (rf_merged_df['BUILT_YR'] < 1900), 'BUILT_DECADE'] = "1890's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1900) & (rf_merged_df['BUILT_YR'] < 1910), 'BUILT_DECADE'] = "1900's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1910) & (rf_merged_df['BUILT_YR'] < 1920), 'BUILT_DECADE'] = "1910's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1920) & (rf_merged_df['BUILT_YR'] < 1930), 'BUILT_DECADE'] = "1920's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1930) & (rf_merged_df['BUILT_YR'] < 1940), 'BUILT_DECADE'] = "1930's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1940) & (rf_merged_df['BUILT_YR'] < 1950), 'BUILT_DECADE'] = "1940's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1950) & (rf_merged_df['BUILT_YR'] < 1960), 'BUILT_DECADE'] = "1950's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1960) & (rf_merged_df['BUILT_YR'] < 1970), 'BUILT_DECADE'] = "1960's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1970) & (rf_merged_df['BUILT_YR'] < 1980), 'BUILT_DECADE'] = "1970's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1980) & (rf_merged_df['BUILT_YR'] < 1990), 'BUILT_DECADE'] = "1980's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 1990) & (rf_merged_df['BUILT_YR'] < 2000), 'BUILT_DECADE'] = "1990's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 2000) & (rf_merged_df['BUILT_YR'] < 2010), 'BUILT_DECADE'] = "2000's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 2010) & (rf_merged_df['BUILT_YR'] < 2020), 'BUILT_DECADE'] = "2010's"
rf_merged_df.loc[(rf_merged_df['BUILT_YR'] >= 2020) & (rf_merged_df['BUILT_YR'] < 2030), 'BUILT_DECADE'] = "2020's"

rf_merged_df.to_csv(os.path.join(outputs, 'davis_housing_unit_inventory.csv'), index=False)
# rf_merged_df.to_csv(os.path.join(outputs, 'davis_housing_unit_inventory_sub.csv'), index=False)